<a href="https://colab.research.google.com/github/start94/Automatic_synthesis_of_medical_records/blob/main/Sintesi_automatica__di_cartelle__cliniche.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch requests



# PRESENTAZIONE DEL PROGETTO
Il progetto costruisce un sistema automatico in grado di:
Leggere cartelle cliniche da un file JSON (via URL)
Generare un riassunto automatico con un modello NLP pre-addestrato (BART), salvare i risultati in un file clinical_summaries.json
stampare i riassunti e alcune statistiche finali

Importazione delle librerie:
transformers: usato per caricare il modello BART (da HuggingFace).
torch: per gestire i tensori e l'esecuzione su GPU.
warnings: per disattivare avvisi non critici.
datetime: per aggiungere un timestamp alla generazione.

Caricamento del modello
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)
si carica BART, un modello di summarization.
si controlla se è disponibile la GPU con torch.cuda.is_available() e si trasferisce lì il modello. Per rendere il processo più veloce

Funzione di preparazione del testo clinico
def prepare_clinical_text(patient_data):
Questa funzione prepara un testo coerente per ogni paziente, combinando:
Nome del paziente
ricoveri con date, diagnosi, anamnesi, prognosi
Farmaci somministrati e risultati di esami
 Serve per dare al modello un contesto completo da cui generare un riassunto.

Funzione di generazione del riassunto
def generate_summary(text, max_length=150, min_length=50):
Il testo viene tokenizzato (convertito in numeri).
il modello BART genera un riassunto.
il risultato viene decodificato in linguaggio naturale.
Parametri:
max_length, min_length: controllano la lunghezza del riassunto.
num_beams=4: beam search per migliorare la qualità.

Funzione per caricare i dati clinici
def load_clinical_data(url: str):
Scarica un file .json contenente le cartelle cliniche da un URL remoto.

Gestisce gli errori di rete o formattazione JSON.
Ritorna una lista di pazienti con le loro informazioni cliniche.
Caricamento dei dati dal file JSON
json_url = "..."
clinical_data = load_clinical_data(json_url)
carica i dati dalla URL specificata.
Se il file non è accessibile, mostra un errore e inizializza una lista vuota.

Generazione dei riassunti
ciclo for
for patient in clinical_data:
itera ogni paziente.


Salvataggio dei risultati

with open('clinical_summaries.json', 'w') as f:
I risultati vengono salvati in un file JSON.
Contiene tutti i riassunti e le informazioni di contesto:
Data di generazione
modello usato
numero di pazienti

Visualizzazione dei riassunti
stampa ogni riassunto generato con:
nome del paziente
numero di ricoveri
lunghezza del testo originale
riassunto generato

Statistiche finali
print(f"Total hospitalizations: ...")
Mostra:
numero totale di pazienti processati
numero totale di ricoveri
lunghezza media del testo clinico
modello utilizzato
dispositivo (GPU/CPU)
Conclusione
Il sistema costruito è:
Riproducibile (usa dati esterni da URL)
Scalabile (funziona per molti pazienti)
Realistico (simula un'applicazione in ambito ospedaliero)
Basato su NLP: sfrutta modelli pre-addestrati di HuggingFace per NLP in italiano o inglese.



In [ ]:
import json
import torch
from transformers import BartForConditionalGeneration, BartTokenizer
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


print("🏥 Automatic clinical record synthesis system")


# 1. MODEL LOADING (HUGGINGFACE)
print("Loading BART model")
model_name = "facebook/bart-large-cnn" # specifica il modello
tokenizer = BartTokenizer.from_pretrained(model_name)  #carica il tokenizzatore associato al modello
model = BartForConditionalGeneration.from_pretrained(model_name)


# Controlla se è disponibile una GPU e sposta il modello su di essa per essere più veloce
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(f"Model loaded on: {device}")

# 2. FUNCTION FOR TEXT PREPARATION
def prepare_clinical_text(patient_data):
    """
    Prepares the clinical text for each patient by concatenating all the relevant information
    from their hospitalizations. This creates a comprehensive input for the summarization model.
    """
    patient_name = patient_data['patient_name']
    text_parts = [f"Patient: {patient_name}"]

    # cilco for per scorrere ogni cartella di ricovero del paziente
    for i, hospitalization in enumerate(patient_data['hospitalizations'], 1):
        admission_date = hospitalization['admission_date']
        discharge_date = hospitalization['discharge_date']
        diagnosis = hospitalization['diagnosis']
        anamnesis = hospitalization['anamnesis']
        prognosis = hospitalization['prognosis']

        # costruisco il testo per l'attuale ricovero ospedaliero
        hospitalization_text = f"Hospitalization {i}: From {admission_date} to {discharge_date}. "
        hospitalization_text += f"Diagnosis: {diagnosis}. "
        hospitalization_text += f"Anamnesis: {anamnesis}. "
        hospitalization_text += f"Prognosis: {prognosis}. "

        # aggiungere farmaci se disponibili
        if hospitalization['medications']:
            medications = ", ".join(hospitalization['medications'])
            hospitalization_text += f"Medications administered: {medications}. "


# condizione che aggiunge i risultati del test se disponibili
        if hospitalization['test_results']:
            test_results = ". ".join(hospitalization['test_results'])
            hospitalization_text += f"Test results: {test_results}. "

        text_parts.append(hospitalization_text)

    return " ".join(text_parts)

# 3. FUNCTION FOR SUMMARY GENERATION
def generate_summary(text, max_length=150, min_length=50):
    """
    Ecco cosa fa questa funzione :
    Genera un riepilogo utilizzando il modello BART.
    tokenizza il testo di input: convertilo in ID numerici comprensibili dal modello.
    max_length=1024: limita la lunghezza della sequenza di input per evitare di superare la capacità del modello.
    truncation=True: tronca il testo se è più lungo di max_length.
    """

    inputs = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)
    inputs = inputs.to(device)

    # genera il riepilogo utilizzando il modello in modalità di valutazione,nessun calcolo del gradiente
    with torch.no_grad():
        summary_ids = model.generate(
            inputs,
            max_length=max_length,    # lunghezza massima del riepilogo generato
            min_length=min_length,    # lunghezza min.
            length_penalty=2.0,       # penalità per i riassunti brevi, incoraggia quelli più lunghi
            num_beams=4,              # migliora la qualità del riepilogo
            early_stopping=True       # qui invece interropmppo  la generazione quando viene prodotto un token finale corretto
        )

    """
    skip_special_tokens=True
    Indica di saltare i token speciali come <pad>, <s>, </s>, [CLS], [SEP] che servono al modello ma non devono apparire nel testo finale.
    Senza questo parametro, il riassunto potrebbe contenere simboli inutili o confusi per chi leggee.
    """

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

#  FUNCTION TO LOAD CLINICAL DATA
import requests
def load_clinical_data(url: str) -> list or None:
    """
Carica i dati clinici da un URL specificato.
Argomenti: url (str): URL del file JSON contenente le cartelle cliniche.
Restituisce: elenco o Nessuno: il contenuto del file JSON come elenco di dizionari,
oppure nessuno se si verifica un errore durante il download o l'analisi.
    """
    try:
        print(f" Loading data from URL: {url}")
        response = requests.get(url)
        response.raise_for_status()
        data = json.loads(response.text)
        return data
    except requests.exceptions.RequestException as e:
        print(f" Network/HTTP error during URL loading: {e}")
        return None
    except json.JSONDecodeError:
        print(f" Error: The content downloaded from '{url}' is not valid JSON.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred while loading data: {e}")
        return None

#  LOADING CLINICAL DATA
print("\n Loading clinical data.")

# jsonfile da url perferisco così e non da locale
json_url = "https://raw.githubusercontent.com/Profession-AI/progetti-deeplearning/refs/heads/main/Sintesi%20automatica%20di%20cartelle%20cliniche%20di%20un%E2%80%99azienda%20ospedaliera/hospital_records.json"
clinical_data = load_clinical_data(json_url)

# piccolo if controlla se i dati sono stati caricati correttamente, in caso contrario, stampa un errore
if clinical_data is None:
    print(" Failed to load data. Using example data.")
    clinical_data = []
else:
    print(f" Loaded {len(clinical_data)} patients from the JSON file")

# PROCESSING AND GENERATING SUMMARIES
if clinical_data is None or len(clinical_data) == 0:
    print(" No data available for processing. Exiting.")
    exit()

print("\n Generating clinical summaries.")
summaries = [] # elenco per memorizzare i riepiloghi generati per tutti i pazienti

# cilco for per scorrere la cartella di ogni paziente per generare un riepilogo
for i, patient in enumerate(clinical_data, 1):
    print(f"Processing {patient['patient_name']} ({i}/{len(clinical_data)})")

    # preparare il testo clinico concatenando le informazioni rilevanti
    clinical_text = prepare_clinical_text(patient)

    # genera il riepilogo utilizzando il modello BART
    summary = generate_summary(clinical_text)
    patient_summary = {
        "patient_name": patient['patient_name'],
        "original_text_length": len(clinical_text),
        "summary": summary,
        "total_hospitalizations": len(patient['hospitalizations'])
    }

    summaries.append(patient_summary)

# SAVING RESULTS
print("\n Saving results.")

output_data = {
    "generation_date": datetime.now().isoformat(),  #data e ora della generazione del file, in formato leggibile
    "model_used": model_name,                       #nome del modello NLP utilizzato
    "total_patients": len(clinical_data),           # numero totale di pazienti elaborati
    "summaries": summaries                          #lista dei riassunti generati, uno per ciascun paziente
}

# poi adesso in questo piccolo blocco salvo i risult<ti in json
try:
    with open('clinical_summaries.json', 'w', encoding='utf-8') as f:
        json.dump(output_data, f, indent=2, ensure_ascii=False)
    print(" File 'clinical_summaries.json' saved successfully!")
except IOError as e:
    print(f" Error saving output file: {e}")

#DISPLAYING RESULTS
print("\n📊 GENERATED RESULTS:")


for summary_item in summaries:
    print(f"\n{summary_item['patient_name']}")
    print(f" Original text length: {summary_item['original_text_length']} characters")
    print(f"Total hospitalizations: {summary_item['total_hospitalizations']}")
    print(f" Summary: {summary_item['summary']}")


#FINAL STATISTICS
print("\n📈 FINAL STATISTICS:")
# calcola i ricoveri totali di tutti i pazienti elaborati
total_hospitalizations = sum(s['total_hospitalizations'] for s in summaries)
#calcola la lunghezza media del testo originale
avg_text_length = sum(s['original_text_length'] for s in summaries) / len(summaries)

print(f"👥 Patients processed: {len(summaries)}")
print(f"🏥 Total hospitalizations: {total_hospitalizations}")
print(f"📏 Average original text length: {avg_text_length:.0f} characters")
print(f"🤖 Model used: {model_name}")
print(f"💻 Device: {device}")

print("\n Processing completed successfully!")
print("The system is now ready for use in a hospital environment!")

# Raffaele Diomaiuto studente del corso AI Development

🏥 Automatic clinical record synthesis system
Loading BART model
Model loaded on: cuda

 Loading clinical data.
 Loading data from URL: https://raw.githubusercontent.com/Profession-AI/progetti-deeplearning/refs/heads/main/Sintesi%20automatica%20di%20cartelle%20cliniche%20di%20un%E2%80%99azienda%20ospedaliera/hospital_records.json
 Loaded 10 patients from the JSON file

 Generating clinical summaries.
Processing Patient 1 (1/10)
Processing Patient 2 (2/10)
Processing Patient 3 (3/10)
Processing Patient 4 (4/10)
Processing Patient 5 (5/10)
Processing Patient 6 (6/10)
Processing Patient 7 (7/10)
Processing Patient 8 (8/10)
Processing Patient 9 (9/10)
Processing Patient 10 (10/10)

 Saving results.
 File 'clinical_summaries.json' saved successfully!

📊 GENERATED RESULTS:

Patient 1
 Original text length: 515 characters
Total hospitalizations: 2
 Summary: Patient 1: Diagnosis: Pneumonia. Medications administered: Captopril, Prednisone, Ciprofloxacin, Ibuprofen. Patient 2: CT scan shows infla